Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [4]:
df = pd.DataFrame()
lr =0.4
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    cluster_number = np.unique(Y).shape[0]
    nb_genes = 500
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    X = train.preprocess(X, nb_genes=nb_genes)
    for temperature in tqdm([0.07]):
        for dropout in [ 0.9
                       ]:
            for noise in [0, 0.01, 0.1, 1
                      ]:
                for layers in [
                    [200, 40, 60]
                              ]:
                    for run in range(3):
                        torch.manual_seed(run)
                        torch.cuda.manual_seed_all(run)
                        np.random.seed(run)
                        random.seed(run)
                        torch.backends.cudnn.deterministic = True
                        torch.backends.cudnn.benchmark = False
                        dresults = train.run(X,
                                             cluster_number,
                                             dataset,
                                             Y=Y,
                                             nb_epochs=30,
                                             lr=lr,
                                             noise = noise,
                                             temperature=temperature,
                                             dropout=dropout,
                                             evaluate=False,
                                             n_ensemble=1,
                                             layers=layers,
                                             save_to=f"{path}output/real_data/{run}/",
                                             save_pred = False)
                        dresults["temperature"] = temperature
                        dresults["dropout"] = dropout
                        dresults["noise"] = noise
                        dresults["layers"] = str(layers)
                        dresults["run"] = run
                        print(f".", end = "")
                        print(f"# {temperature}, {dropout}, {lr}, {layers}", 
                              dresults.get('COMBINED_kmeans_ari', ""),
                              dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
                              dresults.get('leiden_ari_0', ""))
                        df = df.append(dresults, ignore_index=True)

                        df.to_pickle(f"{path}output/pickle_results/real_data/real_data_noise.pkl")

>>>>> Data Quake_Smart-seq2_Trachea
SCZI  0.8291128219663276
(1350, 23341) (1350, 23341) keeping 500 genes


.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8757419746828033 0.18813728466297122
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8231397433913237 0.1741258162593457
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8804301360194373 0.17054375955346077
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8312820445723751 0.20125822480581984
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8625107505389255 0.181046818927349
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8832595550526786 0.20281936397504144
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8096524278000525 0.1873295303767348
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8600485254309981 0.18645554277253612
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8272110945798262 0.1781162525838478
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6893990113800084 0.17431489355449595
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6891170725807442 0.19037254779769372
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7026631411080548 0.1986611610276998

>>>>> Data Quake_Smart-seq2_Diaphragm
SCZI  0.9596701189611787
(870, 23341) (870, 23341) keep

.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9697968135271604 0.3499212283088658
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9660890110636096 0.38953113201131523
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.966116503913852 0.28111962100728
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9815350470962564 0.34473765055160915
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9799145967962042 0.3413454780632075
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9762682613429001 0.3361930630622387
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9671067930123277 0.3210503998920907
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9816971648981092 0.3072719861724049
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9676047817831751 0.33941183702965533
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9138256289398574 0.3197802751332067
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.4526373764458337 0.2893523502442978
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9022120281211342 0.36384316503781766

>>>>> Data Quake_10x_Spleen
SCZI  0.909534824400645
(9552, 23341) (9552, 23341) keeping 500 genes


.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9179554666263199 0.10285435216195174
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9058783786676112 0.093658663050477
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8935893555591153 0.08174096312730321
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5393296881135834 0.09510186266113792
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9180586807842281 0.09200141870909279
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9134964286384969 0.1067318634092067
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9036412870786968 0.09870121643689628
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9157860190334813 0.09606894113108683
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9267398926019649 0.07827918539826322
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9318820141298187 0.09328187781597455
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9404004820561515 0.08868294261426181
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9356784445375036 0.1032637392535893

>>>>> Data Young
SCZI  0.6629811886776039
(5685, 33658) (5685, 33658) keeping 500 genes


.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6119281116035161 0.4743239647406442
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5908839137673562 0.4423540102052737
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5842727421250747 0.43005019653166804
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6001641972918926 0.44869875805177617
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5913627010685464 0.4190265454070551
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6069854881850048 0.44520084078863825
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6071546234110726 0.4356931369468819
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5962769125034093 0.4261110915020897
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6160817205365662 0.454908282837452
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.45987958976612797 0.33231677504492724
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.3999283027256634 0.3133025399660646
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5651000881564101 0.37037658060689543

>>>>> Data mouse_ES_cell
SCZI  0.800376096099337
(2717, 24175) (2717, 24175) keeping 500 genes

...

.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.4159062548119207 0.3630343188812175
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.33583684457862745 0.3376944609658503
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.36819192256785693 0.33873315844672597
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.32595268125942745 0.3279410476248205
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.33473527548112636 0.32639346089431276
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.335603149317031 0.3438785298929182
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.33273859851347704 0.31607791598744794
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.34635781042382857 0.3728423171598815
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.3636946201541781 0.28874055732097426
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.29222179650246033 0.2642150609165803
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.3451851590167957 0.27949433486766306
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.3407840596250408 0.2764328928458733

>>>>> Data Romanov
SCZI  0.75405351856634
(2881, 21143) (2881, 21143) keeping 500 genes


.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7284388899982704 0.2824650881563508
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7354186431860008 0.28504750448831334
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7289856480038408 0.2581196957117909
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6965691391904091 0.23094782044211934
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7058386454845831 0.28705975831105485
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7327421896398879 0.2999857416831402
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7383805621333666 0.27238033501440134
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.734201428225214 0.27205001678014096
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6617031128163018 0.28435314483144003
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.39882392806401096 0.16386843181549643
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.3867689578142621 0.20866726546293157
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.3874333198839162 0.21649286140337484

>>>>> Data Quake_Smart-seq2_Limb_Muscle
SCZI  0.9736759009464299
(1090, 23341) (1090, 2334

.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9722957002915682 0.35379376911093435
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9733778655307548 0.31004603378308343
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9739575927457615 0.3618604627919782
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.968832897043171 0.3815898184333683
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9758813374991732 0.31057102294381056
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9714047000561042 0.345643647933032
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9707162109190607 0.36030094208591035
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9765941773588556 0.35571967982882674
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9736885752148925 0.2755242336194279
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9335013636218884 0.32421023250909164
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.5172909135496863 0.31160206602117774
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.9156352209707112 0.31080144358559764

>>>>> Data Muraro
SCZI  0.7246271034758398
(2122, 19046) (2122, 19046) keeping 500 genes


.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8522487093977664 0.3774794154209196
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8610519908167902 0.37833295995145977
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.878946946231079 0.372726984708388
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8356180405479354 0.3501495439749779
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.840017554672249 0.3449483840837134
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6613934943934656 0.36377261063807736
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.8693392050017619 0.3886752383776422
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7512850030771876 0.31572325638080606
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7082351735371902 0.3904935579832978
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.34188728023035464 0.2529884481096396
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.33698299619770833 0.2302584951700248
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.33860258460499276 0.20568018926696174

>>>>> Data 10X_PBMC
SCZI  0.5845649506030623
(4271, 16653) (4271, 16653) keeping 500 genes


.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7000465730202853 0.38195140087809054
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7012140678789418 0.39608507554978317
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.703442341359809 0.44127981801223143
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6975008154164312 0.38706377606690073
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6915768652596332 0.3825626090357679
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7111128897658276 0.3583043963452998
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6874305088375064 0.3990753568162678
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7096820493178939 0.40231870569670625
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.7129863799367051 0.3863111738067699
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6072964659002577 0.3357633149277467
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6027552872583546 0.2992224650110242
.# 0.07, 0.9, 1e-05, [200, 40, 60]   0.6516874386050853 0.31638920550065214



In [ ]:
df

In [ ]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")
sczi = sczi[sczi["dataset"].isin(files)]
sczi.mean()

In [ ]:
df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_noise.pkl")
df = df.fillna(0)
df = df[df["dataset"].isin(files)]
df.groupby(["temperature", "layers", "dropout", "noise"])["kmeans_ari_0"].mean().unstack(["layers", "noise"]).T

In [ ]:
# df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "noise"]).max()

In [ ]:
# r = df[(df["layers"]=="[40]")
#        &(df["temperature"]==0.01)
#        &(df["lr"]==1e-5)
#        &(df["dropout"]==0.9)
#       ]
r = df[(df["layers"]=="[200, 100, 30, 30]")
       &(df["temperature"]==0.07)
       &(df["lr"]==1e-5)
       &(df["dropout"]==0.9)
      ]


r.mean()

In [ ]:
comb = pd.merge(r, sczi, on=["dataset", "run"])[[
    "dataset", "kmeans_ari_0", "ARI", "kmeans_nmi_0", "NMI"
]].rename(columns = {"kmeans_ari_0": "contrative-sc", "ARI": "sczi"})
comb

In [ ]:
comb =pd.melt(comb, id_vars=["dataset"], value_vars=["contrative-sc", "sczi"])

In [ ]:
comb.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot( data = comb, y="value", x = "variable")
plt.xticks(rotation = 90)

In [ ]:
sns.barplot(x = "dataset", data = comb, y="value", hue = "variable")
plt.xticks(rotation = 90)

In [ ]:
df.groupby("dataset").mean()

# Importance of input size

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_genes", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    for run in range(2):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        for nb_genes in [100, 200, 500, 1000, 1500, 2000, 5000, 8000]:

            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            cluster_number = np.unique(Y).shape[0]

            X = train.preprocess(X, nb_genes=nb_genes)
            nb_zeros = int(0.8 * nb_genes)
            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=100,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/inputs/{dataset}_{nb_genes}/")

            #         df.loc[df.shape[0]] = [
            #                 dataset, perc_0, nb_genes, 'kmeans_representation_0',dresults['kmeans_representation_0']
            #             ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, nb_genes, 'leiden_representation_0',
                dresults['leiden_representation_0'], run]

#             pxt = PCA(2).fit_transform(X)
#             dresults["original"] = utils.evaluate(X, Y, cluster_number)[1]
#             dresults["pca"] = utils.evaluate(pxt, Y, cluster_number)[1]
            print(dresults)
    df.to_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df.groupby(["nb_genes"]).mean()

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_genes": "Nb input genes"})

In [ ]:
df["dataset"].unique()


In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size.pdf", bbox_inches='tight')

In [ ]:
datasets = ['10X PBMC (2100)',
       'Mouse ES\nCell (2100)', 'Worm Neuron\nCell (2100)',
       'Mouse Bladder\n Cell (2100)']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size_2100.pdf", bbox_inches='tight')

# Importance of the number of epochs

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_epochs", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    nb_genes = 1500
    for epochs in [5, 50, 100, 300]:

        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
        print(f"Perc 0 {perc_0}")
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)
        nb_zeros = int(0.8 * nb_genes)
        for run in range(2):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=epochs,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/epochs/{dataset}_{epochs}/")

            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'kmeans_representation_0',
                dresults['kmeans_representation_0'], run
            ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'leiden_representation_0',
                dresults['leiden_representation_0'], run
            ]

            print(dresults)
            df.to_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_epochs": "Nb epochs"})

In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(7, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb epochs",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title ="Number of epochs",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_nb_epochs.pdf", bbox_inches='tight')